# Copy a Pretrained Network between Frameworks

Since a large CNN is very time-consuming to train (even on a GPU), and requires huge amounts of data, is there any way to use a pre-calculated one instead of retraining the whole thing from scratch?  

This notebook shows how this can be done, so that the same data can be used in a different framework.

The code here is slightly rough-and-ready, since to be interested in doing it assumes some level of familiarity...

In [ ]:
import tensorflow as tf
import numpy as np

### Add TensorFlow Slim Model Zoo to path

In [ ]:
import os, sys
better_instructions = '2-CNN/4-ImageNet/4-ImageClassifier-inception_tf.ipynb'

if not os.path.isfile( '../models/tensorflow_zoo/models/README.md' ):
    print("Please follow the instructions in %s to get the Slim-Model-Zoo installed" % better_instructions)
else:
    sys.path.append(slim_models_dir + "/models/slim")
    print("Model Zoo model code installed")

In [ ]:
from datasets import dataset_utils

checkpoint_file = '../data/tensorflow_zoo/checkpoints/inception_v1.ckpt'
if not os.path.isfile( checkpoint_file ):
    print("Please follow the instructions in %s to get the Checkpoint installed" % better_instructions)
else:
    print("Checkpoint available locally")

In [ ]:
if not os.path.isfile('../data/imagenet_synset_words.txt'):
    print("Please follow the instructions in %s to get the synset_words file" % better_instructions)
else:    
    print("ImageNet synset labels available")

### Build the model in TensorFlow

In [ ]:
slim = tf.contrib.slim
from nets import inception
#from preprocessing import inception_preprocessing

#image_size = inception.inception_v1.default_image_size
#image_size

In [ ]:
tf.reset_default_graph()

if False:
    # Define the pre-processing chain within the graph - from a raw image
    input_image = tf.placeholder(tf.uint8, shape=[None, None, None, 3], name='input_image')
    processed_image = inception_preprocessing.preprocess_image(input_image, image_size, image_size, is_training=False)
    processed_images = tf.expand_dims(processed_image, 0)

processed_images = tf.placeholder(tf.float32, shape=[None, None, None, 3], name='input_image')

# Create the model - which uses the above pre-processing on image
#   it also uses the default arg scope to configure the batch norm parameters.
print("Model builder starting")

# Here is the actual model zoo model being instantiated :
with slim.arg_scope(inception.inception_v1_arg_scope()):
    logits, _ = inception.inception_v1(processed_images, num_classes=1001, is_training=False)
#probabilities = tf.nn.softmax(logits)

# Create an operation that loads the pre-trained model from the checkpoint
init_fn = slim.assign_from_checkpoint_fn(checkpoint_file, slim.get_model_variables('InceptionV1') )

print("Model defined")

### Get the values from the TF model into a NumPy structure
Mostly because it's easier for me to reason about NumPy...

In [ ]:
capture_names =[] 
capture_values=dict()

# Now let's run the pre-trained model
with tf.Session() as sess:
    # This is the loader 'op' we defined above
    init_fn(sess)  
    
    # This is two ops : one merely loads the image from numpy, 
    #   the other runs the network to get the class probabilities
    #np_image, np_probs = sess.run([numpyish_image, probabilities], feed_dict={input_image:im_sq})
    
    variables = tf.trainable_variables()
    #variables = tf.model_variables()  # includes moving average information
    for variable in variables:
        name, value = variable.name, variable.eval()
        capture_names.append(name)
        capture_values[name] = value
        print("%20s %s " % (value.shape, name, ))

----------
## Define the Model Structure in Framework 'B'

( choosing Keras here )


In [ ]:
# This is taken from https://github.com/fchollet/deep-learning-models/blob/master/inception_v3.py
from keras.models import Model
from keras import layers
from keras.layers import Activation
from keras.layers import Dense
from keras.layers import Input
from keras.layers import BatchNormalization
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import AveragePooling2D
from keras.layers import GlobalAveragePooling2D
from keras.layers import GlobalMaxPooling2D
from keras.engine.topology import get_source_inputs
from keras.utils.layer_utils import convert_all_kernels_in_model
from keras.utils.data_utils import get_file
from keras import backend as K
from keras.applications.imagenet_utils import decode_predictions
from keras.applications.imagenet_utils import _obtain_input_shape
from keras.preprocessing import image

In [ ]:
# This is taken from https://github.com/fchollet/deep-learning-models/blob/master/inception_v3.py
def conv2d_bn(x,
              filters,
              num_row,
              num_col,
              padding='same',
              strides=(1, 1),
              normalizer=True,
              activation='relu',
              name=None):
    """Utility function to apply conv + BN.
    Arguments:
        x: input tensor.
        filters: filters in `Conv2D`.
        num_row: height of the convolution kernel.
        num_col: width of the convolution kernel.
        padding: padding mode in `Conv2D`.
        strides: strides in `Conv2D`.
        name: name of the ops; will become `name + '_conv'`
            for the convolution and `name + '_bn'` for the
            batch norm layer.
    Returns:
        Output tensor after applying `Conv2D` and `BatchNormalization`.
    """
    if name is not None:
        bn_name = name + '_bn'
        conv_name = name + '_conv'
    else:
        bn_name = None
        conv_name = None
    if K.image_data_format() == 'channels_first':
        bn_axis = 1
    else:
        bn_axis = 3
    x = Conv2D(
        filters, (num_row, num_col),
        strides=strides,
        padding=padding,
        use_bias=False,
        name=conv_name)(x)
    if normalizer:
        x = BatchNormalization(axis=bn_axis, scale=False, name=bn_name)(x)
    if activation:
        x = Activation(activation, name=name)(x)
    return x

In [ ]:
# Convenience function for 'standard' Inception concatenated blocks
def concatenated_block(x, specs, channel_axis, name):
    (b0, b1, b2, b3) = specs   # ((64,), (96,128), (16,32), (32,))
    
    branch_0 = conv2d_bn(x, b0[0], 1, 1)

    branch_1 = conv2d_bn(x, b1[0], 1, 1)
    branch_1 = conv2d_bn(branch_1, b1[1], 3, 3)

    branch_2 = conv2d_bn(x, b2[0], 1, 1)
    branch_2 = conv2d_bn(branch_2, b2[1], 3, 3)

    branch_3 = MaxPooling2D( (3, 3) )(x)  
    branch_3 = conv2d_bn(branch_3, b3[0], 1, 1)

    x = layers.concatenate(
        [branch_0, branch_1, branch_2, branch_3,],
        axis=channel_axis,
        name=name)
    return x

In [ ]:
def InceptionV1(include_top=True,
                weights='imagenet',
                input_tensor=None,
                input_shape=None,
                pooling=None,
                classes=1000):
    """Instantiates the Inception v1 architecture.

    This architecture is defined in:
        Going deeper with convolutions
        Christian Szegedy, Wei Liu, Yangqing Jia, Pierre Sermanet, Scott Reed,
        Dragomir Anguelov, Dumitru Erhan, Vincent Vanhoucke, Andrew Rabinovich.
        http://arxiv.org/pdf/1409.4842v1.pdf.
    
    Optionally loads weights pre-trained
    on ImageNet. Note that when using TensorFlow,
    for best performance you should set
    `image_data_format="channels_last"` in your Keras config
    at ~/.keras/keras.json.
    The model and the weights are compatible with both
    TensorFlow and Theano. The data format
    convention used by the model is the one
    specified in your Keras config file.
    Note that the default input image size for this model is 224x224.
    Arguments:
        include_top: whether to include the fully-connected
            layer at the top of the network.
        weights: one of `None` (random initialization)
            or "imagenet" (pre-training on ImageNet).
        input_tensor: optional Keras tensor (i.e. output of `layers.Input()`)
            to use as image input for the model.
        input_shape: optional shape tuple, only to be specified
            if `include_top` is False (otherwise the input shape
            has to be `(224, 224, 3)` (with `channels_last` data format)
            or `(3, 224, 224)` (with `channels_first` data format).
            It should have exactly 3 inputs channels,
            and width and height should be no smaller than 139.
            E.g. `(150, 150, 3)` would be one valid value.
        pooling: Optional pooling mode for feature extraction
            when `include_top` is `False`.
            - `None` means that the output of the model will be
                the 4D tensor output of the
                last convolutional layer.
            - `avg` means that global average pooling
                will be applied to the output of the
                last convolutional layer, and thus
                the output of the model will be a 2D tensor.
            - `max` means that global max pooling will
                be applied.
        classes: optional number of classes to classify images
            into, only to be specified if `include_top` is True, and
            if no `weights` argument is specified.
    Returns:
        A Keras model instance.
    Raises:
        ValueError: in case of invalid argument for `weights`,
            or invalid input shape.
    """
    if weights not in {'imagenet', None}:
        raise ValueError('The `weights` argument should be either '
                         '`None` (random initialization) or `imagenet` '
                         '(pre-training on ImageNet).')

    if weights == 'imagenet' and include_top and classes != 1000:
        raise ValueError('If using `weights` as imagenet with `include_top`'
                         ' as true, `classes` should be 1000')

    # Determine proper input shape
    input_shape = _obtain_input_shape(
        input_shape,
        #default_size=299,
        default_size=224,
        min_size=139,
        data_format=K.image_data_format(),
        include_top=include_top)

    if input_tensor is None:
        img_input = Input(shape=input_shape)
    else:
        img_input = Input(tensor=input_tensor, shape=input_shape)

    if K.image_data_format() == 'channels_first':
        channel_axis = 1
    else:
        channel_axis = 3

    # 'Sequential bit at start'
    
    # 'Conv2d_1a_7x7'
    x = conv2d_bn(img_input, 64, 7, 7, strides=(2, 2), padding='valid')  

    # 'MaxPool_2a_3x3'
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)  
    
    
    # 'Conv2d_2b_1x1'
    x = conv2d_bn(x,  64, 1, 1, strides=(1, 1), padding='valid')  
    
    # 'Conv2d_2c_3x3'
    x = conv2d_bn(x, 192, 3, 3, strides=(1, 1), padding='valid')  

    # 'MaxPool_3a_3x3'
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)  


    # This the 'Mixed_3b' concatenated block
    x = concatenated_block(x, (( 64,), ( 96,128), (16, 32), ( 32,)), channel_axis, 'Mixed_3b')

    # This the 'Mixed_3c' concatenated block
    x = concatenated_block(x, ((128,), (128,192), (32, 96), ( 64,)), channel_axis, 'Mixed_3c')

    
    # 'MaxPool_4a_3x3'
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)  

    
    # This the 'Mixed_4b' concatenated block
    x = concatenated_block(x, ((192,), ( 96,208), (16, 48), ( 64,)), channel_axis, 'Mixed_4b')
    
    # This the 'Mixed_4c' concatenated block
    x = concatenated_block(x, ((160,), (112,224), (24, 64), ( 64,)), channel_axis, 'Mixed_4c')
    
    # This the 'Mixed_4d' concatenated block
    x = concatenated_block(x, ((128,), (128,256), (24, 64), ( 64,)), channel_axis, 'Mixed_4d')
    
    # This the 'Mixed_4e' concatenated block
    x = concatenated_block(x, ((112,), (144,288), (32, 64), ( 64,)), channel_axis, 'Mixed_4e')

    # This the 'Mixed_4f' concatenated block
    x = concatenated_block(x, ((256,), (160,320), (32,128), (128,)), channel_axis, 'Mixed_4f')

    
    # 'MaxPool_5a_2x2'
    x = MaxPooling2D((2, 2), strides=(2, 2))(x)  


    # This the 'Mixed_5b' concatenated block
    x = concatenated_block(x, ((256,), (160,320), (32,128), (128,)), channel_axis, 'Mixed_5b')
    
    # This the 'Mixed_5c' concatenated block
    x = concatenated_block(x, ((384,), (192,384), (48,128), (128,)), channel_axis, 'Mixed_5c')
    

    if include_top:
        # Classification block
        
        #x = GlobalAveragePooling2D(name='avg_pool')(x)
        # Drop out layer goes here?
        #x = Dense(classes, activation='softmax', name='predictions')(x)
        
        # 'AvgPool_0a_7x7'
        x = AvPooling2D((7, 7), strides=(1, 1))(x)  
        
        # 'Dropout_0b'
        # ??
        
        #x == softmax( logits )
        logits = conv2d_bn(x,  num_classes, 1, 1, strides=(1, 1), padding='valid', normalizer=False, activation=None)  
        x = Activation('softmax', name='predictions')(logits)
    else:
        if pooling == 'avg':
            x = GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            x = GlobalMaxPooling2D()(x)

    # Ensure that the model takes into account
    # any potential predecessors of `input_tensor`.
    if input_tensor is not None:
        inputs = get_source_inputs(input_tensor)
    else:
        inputs = img_input
        
    # Create model.
    model = Model(inputs, x, name='inception_v1')
    
    """ Slim model ::

    """